In [ ]:
import os
from datasets import load_dataset
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

ModuleNotFoundError: No module named 'torch'

In [ ]:
# ssh student@165.132.172.93
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Move the model to the CPU
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)  # Use CPU

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [26]:
# Load Llama 3.1 8B model and tokenizer from Hugging Face
# Load model directly
# Load DROP dataset from Hugging Face
ds = load_dataset("ucinlp/drop", split="train")

# Convert to DataFrame for easier manipulation
df = ds.to_pandas()

# Create a copy of the dataset to avoid modifying the original
df_copy = df.copy()

In [ ]:
def generate_cot(row):
    question = row['question']  # Accessing the 'question' column
    passage = row['passage']  # Accessing the 'passage' column

    # Construct the prompt to generate CoT reasoning
    prompt = f"""
You are an advanced reasoning model designed to generate a coherent, structured, and logical step-by-step reasoning process to answer a question based on the information provided in a passage.

### Instructions:
1. Break down the reasoning into a logical sequence of no more than 10 steps.
2. For each step, create a specific sub-question related to the passage, followed by a precise answer. Each sub-question should build toward reaching the final answer.
3. Aim for clarity and relevance in each sub-question, ensuring that it logically connects to the previous and subsequent steps.
4. Conclude with a **[FINAL ANSWER]** that integrates the information from all previous steps to answer the main question accurately.

### Input Information:
- **Passage:** {passage}
- **Question:** {question}

### Required Output Format:
- **[SUB_QUESTION 1]:** Provide a meaningful first sub-question directly related to understanding the passage.
- **[SUB_SOLUTION 1]:** Give a clear and specific solution to the first sub-question.
- **[SUB_QUESTION 2]:** Continue with a logical next sub-question building upon the prior solution.
- **[SUB_SOLUTION 2]:** Provide a specific solution to the second sub-question.
- **...**
- **[FINAL ANSWER]:** Synthesize all the information from the steps above to answer the main question concisely.

### Your CoT Response. Poyekhali:
"""
   # Tokenize the input prompt
    response = text_generator(prompt, max_length=512, num_return_sequences=1, temperature=0.1)
    
    # Extract generated content
    generated_content = response[0]["generated_text"]
    return generated_content

In [33]:
# Create a directory to save each chunk’s augmented CSV
output_dir = "augmented_chunks"
os.makedirs(output_dir, exist_ok=True)

# Define chunk size and process each chunk separately
chunk_size = 2
start_chunk = 0  # Set this to the first chunk to process (e.g., 0)
end_chunk = 1   # Set this to the last chunk to process (e.g., 10)
num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size != 0 else 0)

In [34]:
# Process each chunk in the specified range
for chunk_num in range(start_chunk, end_chunk + 1):
    start_idx = chunk_num * chunk_size
    end_idx = min(start_idx + chunk_size, len(df_copy))
    
    # Select the chunk of rows
    chunk = df_copy.iloc[start_idx:end_idx]
    
    # Apply CoT generation to each row in the chunk
    chunk['CoT_Question'] = chunk.apply(generate_cot, axis=1)
    print(chunk[['question', 'CoT_Question']].head())  # Print sample to confirm

    
    # Save the augmented chunk to a CSV file
    output_filename = f"{output_dir}/chunk_{chunk_num}_augmented.csv"
    chunk.to_csv(output_filename, index=False)
    print(f"Saved {output_filename}")

print("All chunks in the specified range have been processed and saved.")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/tmp/ipykernel_2034973/358396174.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['CoT_Question'] = chunk.apply(generate_cot, axis=1)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


                                            question  \
0  How many points did the buccaneers need to tie...   
1          How many field goals did the Lions score?   

                                        CoT_Question  
0  - [SUB_QUESTION 1]: How many points did the bu...  
1  ```python\nimport random\nimport string\n\ndef...  
Saved augmented_chunks/chunk_0_augmented.csv


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

In [ ]:
# To combine all processed chunks into one final DataFrame after completion:
all_chunks = []
for chunk_file in sorted(os.listdir(output_dir)):
    if chunk_file.endswith("_augmented.csv"):
        chunk_path = os.path.join(output_dir, chunk_file)
        chunk_df = pd.read_csv(chunk_path)
        all_chunks.append(chunk_df)

In [ ]:
# Combine all chunks into a single DataFrame (optional)
final_df = pd.concat(all_chunks, ignore_index=True)

In [ ]:
# Add a new column 'CoT_Question' with generated Chain-of-Thought responses in the copy
df_copy['CoT_Question'] = df_copy.apply(generate_cot, axis=1)

# Display a few rows from the modified copy to check
print(df_copy[['question', 'CoT_Question']].head())